# **Instalar dependências**

In [ ]:
!pip install --upgrade google-cloud-aiplatform
!pip install --upgrade google-cloud-storage
!pip install --upgrade kfp
!pip install --upgrade google-cloud-pipeline-components

!python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"

### Colab only: Uncomment the following cell to restart the kernel.

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

# **Configurar project ID**


In [ ]:
PROJECT_ID = "teste"  # @param {type:"string"}
REGION = "teste"  # @param {type: "string"}
# Set the project id
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


# **Gerar UUID**


In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))

#Apenas para garantir unicidade de nomes durante a etapa de testes
UUID = generate_uuid()

# **Autenticar Google Cloud account**


In [ ]:
# ! gcloud auth login

**3. Colab, uncomment and run:**

In [ ]:
from google.colab import auth
auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

# **Criar Cloud Storage bucket**


In [ ]:
BUCKET_URI = f"gs://demo-pipe-2-{PROJECT_ID}-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://demo-pipe-2-elite-dreamer-418223-unique/...


# **Definir Service Account**

In [ ]:
SERVICE_ACCOUNT = ""  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

#### Set service account access for Vertex AI Pipelines

Run the following commands to grant your service account access to read and write pipeline artifacts in the bucket that you created in the previous step -- you only need to run these once per service account.

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

# **Importar bibliotecas e definir constantes**

In [ ]:
import kfp
from typing import NamedTuple
from google_cloud_pipeline_components.types import artifact_types
import google.cloud.aiplatform as aip
from kfp import compiler, dsl
from google.cloud import aiplatform
from google_cloud_pipeline_components.types import artifact_types
from google_cloud_pipeline_components.v1.batch_predict_job import ModelBatchPredictOp as batch_prediction_op
from google_cloud_pipeline_components.v1.model import ModelUploadOp as model_upload_op
from kfp.dsl import importer_node,Input, Metrics, Output,component

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI, location=REGION)

# **Configurar Componentes e Pipeline**

In [ ]:
#Criação de componente batch_prediction_job

@component(
  packages_to_install=["google-cloud-aiplatform","kfp","google-cloud-storage"],
  base_image="python:3.9",
  output_component_file="batch_prediction_job.yaml"
)
def batch_prediction_job(
    project: str,
    location: str,
    model_resource_name: str,
    job_display_name: str,
    gcs_source: str,
    gcs_destination: str,
    machine_type: str,
    starting_replica_count: int = 1,
    max_replica_count: int = 1
):

    from google.cloud import aiplatform

    aiplatform.init(project=project, location=location)

    model = aiplatform.Model(model_resource_name)

    batch_prediction_job = model.batch_predict(
        job_display_name=job_display_name,
        instances_format='jsonl', #json
        gcs_source=gcs_source,
        gcs_destination_prefix=gcs_destination,
        machine_type=machine_type, # must be present
    )
    batch_prediction_job.wait()
    print(batch_prediction_job.display_name)
    print(batch_prediction_job.state)



In [ ]:
# Define variables
PIPELINE_ROOT = "{}/pipeline_root/wine-custom-model-pipe".format(BUCKET_URI)

MODEL_DISPLAY_NAME = f"wine-custom-{UUID}"
BATCH_PREDICTION_DISPLAY_NAME = f"wine-batch-prediction-on-pipelines-model-{UUID}"

# Get model id
MODEL_NAME="demo-mlops-custom-image"
MODEL_ID=!(gcloud ai models list --region=$REGION --filter=display_name=$MODEL_NAME)
MODEL_ID=MODEL_ID[2].split(" ")[0]

model_resource_name = f'projects/{PROJECT_ID}/locations/{REGION}/models/{MODEL_ID}'

GCS_SOURCE= f"{BUCKET_URI}/input/instances.jsonl"
gcs_destination_prefix=f"{BUCKET_URI}/output"

# Create working dir
WORKING_DIR = f"{PIPELINE_ROOT}/{UUID}"

machine_type="n1-standard-2"

MODEL_ID

In [ ]:
@dsl.pipeline(
  # Default pipeline root. You can override it when submitting the pipeline.
  pipeline_root=PIPELINE_ROOT,
  # A name for the pipeline. Use to determine the pipeline Context.
  name="demo-vertex-pipelines-custom-model",
)
def pipeline(
    project: str,
    region: str,
    model_display_name: str,
    job_display_name: str,
    batch_prediction_display_name: str,
    gcs_destination_prefix: str,
    gcs_source: str,
    machine_type: str,
    model_id:str,
):
    import kfp
    from typing import NamedTuple
    from google_cloud_pipeline_components.types import artifact_types
    import google.cloud.aiplatform as aip
    from kfp import compiler, dsl
    from kfp.dsl import component
    from google.cloud import aiplatform
    from google_cloud_pipeline_components.types import artifact_types
    from google_cloud_pipeline_components.v1.batch_predict_job import ModelBatchPredictOp as batch_prediction_op
    from google_cloud_pipeline_components.v1.model import ModelUploadOp as model_upload_op
    from kfp.dsl import importer_node,Input, Metrics, component

    model_resource_name = f'projects/{project}/locations/{region}/models/{model_id}'

    task = batch_prediction_job(
        project=project,
        location=region,
        model_resource_name=model_resource_name,
        job_display_name=job_display_name,
        gcs_source=gcs_source,
        gcs_destination=gcs_destination_prefix,
        machine_type=machine_type)


# **Compilar Pipeline**

In [ ]:
compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="demo-vertex-pipelines-custom-model.json",
)

# **Executar Pipeline**


In [ ]:

job = aiplatform.PipelineJob(
    display_name=MODEL_DISPLAY_NAME,
    template_path="wine-custom-model-pipe.json",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={"project": PROJECT_ID,
                      "region": REGION,
                      "model_display_name":MODEL_DISPLAY_NAME,
                      "job_display_name":BATCH_PREDICTION_DISPLAY_NAME,
                      "batch_prediction_display_name":BATCH_PREDICTION_DISPLAY_NAME,
                      "gcs_destination_prefix":gcs_destination_prefix,
                      "gcs_source":GCS_SOURCE,
                      "machine_type": "n1-standard-2",
                      "model_id":MODEL_ID},
    enable_caching=True,
)

job.run()
